# 작사가 인공지능 만들기

- 실습 노트와 프로젝트를 한 파일에 작성해뒀습니다.

## 0. 기본 준비

### import, 경로설정

In [1]:
import os, re
import numpy as np
import tensorflow as tf

In [2]:
!pwd

/aiffel/aiffel/Exploration/natural_language


In [3]:
path = '/aiffel/aiffel/Exploration/natural_language/'

In [4]:
os.listdir(path)

['[E-04]natural_language.ipynb',
 '.ipynb_checkpoints',
 'lyricist',
 'E-12-4.max-800x600.png']

In [5]:
file_dir = path+'lyricist/data/'

In [6]:
os.listdir(file_dir)

['lyrics', 'shakespeare.txt']

### 예시 파일을 읽어보자

In [7]:
file_path = file_dir+'shakespeare.txt'
with open(file_path, "r") as f:          # 경로의 파일을 읽기모드로 열기. f로 명명
    raw_corpus = f.read().splitlines()    # 라인 단위로 분할. 리스트 형태로 읽어온다?
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [8]:
# enumerate(이뉴머레잍?)은 리스트의 요소와 인덱스 번호를 튜플 형태로 반환한다.
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


## 1.데이터 전처리 

1. 정규표현식으로 코퍼스 정리
2. 정리된 코퍼스를 텐서 데이터로 변환
3. 텐서 데이터를 객체화



### 토큰화(Tokenize)

: 주어진 코퍼스(corpus)에서 토큰(token)이라 불리는 단위로 나누는 작업을 토큰화(tokenization)라고 합니다. 토큰의 단위가 상황에 따라 다르지만, 보통 의미있는 단위로 토큰을 정의합니다.

- 코퍼스를 의미 단위로 나누는 작업


- 보통 단어(+단어구, 유의미한 문자열)단위로 시행한다고 한다.


- 정제되지 않은 코퍼스의 경우 문장 단위로 먼저 토큰화


- 한국어는 토큰화가 난해하다.
> - 교착어의 특성(어절 단위에 조사가 포함)
> - 띄어쓰기가 잘 지켜지지 않음(어렵고, 안 지켜져도 변형이 심하지 않기 때문)


- 1~2를 토큰화 과정으로 볼 수 있는 것 같다.

### 띄어쓰기를 기준으로 토큰화 한다면?

#### 문제가 되는 케이스와 대첵
- 문장부호
> 부호 양 옆에 공백을 추가
- 대/소문자 구분
> 모든 문자를 소문자로 변환
- 특수문자(ten-year-old 등)
> 특수문자는 일단 다 제거

## 1.1. 정규표현식으로 코퍼스 정리 

### 정규표현식(Regex)을 이용한 필터링

In [9]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 \1은 정규표현식 그룹1을 가리킨다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


#### 자연어 처리 용어
- Source Sentence(소스 문장) : 모델의 입력이 되는 문장. X_train에 해당
- Target Sentence(타겟 문장) : 모델의 정답(출력) 문장. y_train에 해당

### 정제된 문장을 리스트화

In [10]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

## 1.2. 정리된 코퍼스를 텐서 데이터로 변환

### 토큰화 코드 : 코퍼스를 텐서로 변환

In [11]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f6743d82490>


### 토큰화 된 텐서 데이터의 인덱스를 확인해보자

* 인덱스 0은 패딩 문자 \<pad>

In [12]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


## 1.3. 텐서 데이터를 객체화

### 텐서를 소스와 타겟으로 분할

In [13]:
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다. (따라서 =1 입력)
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


### 데이터셋 객체 생성
- 지금까지 실습에서 대부분 fit() 메소드를 사용해 npArray 데이터 셋을 생성, 모델에 제공
> model.fit(X_train, y_train) 의 형태

- 텐서플로우에서는 데이터를 텐서 타입으로 생성해서 학습시킨다.
> tf.data.Detaset을 이용

- 여기서는 이미 텐서 형태로 토큰화 했기 때문에 슬라이싱하여 객체를 생성할 것

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

## 2. 학습 시키기

- 실습할 모델의 구조도
![-](./E-12-4.max-800x600.png)

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

- embedding_size : 단어가 추상적으로 표현되는 크기. 학습 관점에서 batch size로 볼 수 있다.


- hidden_size : 모델의 일꾼(두뇌? 판단지표?)수 라고 한다.

In [18]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-3.8165841e-04, -2.6243613e-05, -1.3113992e-04, ...,
          3.4032954e-04,  2.2576917e-04, -3.0043759e-04],
        [-4.2603182e-04,  1.3185639e-04, -1.9268265e-04, ...,
          3.2288401e-04,  2.7062604e-04, -1.7772862e-04],
        [-4.0846737e-04,  1.0529251e-04, -1.5365033e-04, ...,
          2.2728206e-04, -2.0570704e-05, -3.3829674e-05],
        ...,
        [-2.7526354e-03,  2.6553178e-03,  2.3215306e-03, ...,
          1.4744591e-03,  2.1361527e-03,  2.8783174e-03],
        [-3.0750979e-03,  3.0227075e-03,  2.6943702e-03, ...,
          1.5144056e-03,  2.3203588e-03,  3.3736196e-03],
        [-3.3267655e-03,  3.3184418e-03,  3.0302031e-03, ...,
          1.5087611e-03,  2.4791225e-03,  3.7974222e-03]],

       [[-3.8165841e-04, -2.6243613e-05, -1.3113992e-04, ...,
          3.4032954e-04,  2.2576917e-04, -3.0043759e-04],
        [-7.5666863e-04, -3.5306824e-05, -2.9468647e-04, ...,
          4.6486451e-04, -

#### tf.Tensor: shape=(256, 20, 7001)

- 256 : dataset.take(1)으로 1개의 데이터셋(shape=(256, 20))을 가져온 결과

- 7001 : 7001개의 단어에 대한 예측 확률 값

- 20 : ???? 
> 그렇다면 20은 무엇을 의미할까요? 비밀은 바로 tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True이라고 지정한 부분에 있습니다. 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다. 만약 return_sequences=False였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다.
>
>그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모른다는 점입니다. 모델을 만들면서 알려준 적도 없습니다. 그럼 20은 언제 알게된 것일까요? 네, 그렇습니다. 데이터를 입력받으면서 비로소 알게 된 것입니다. 우리 데이터셋의 max_len이 20으로 맞춰져 있었던 것입니다.

In [19]:
model.summary()
#우리의 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없는 것입니다.

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


### 학습!

In [21]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 20s 167ms/step - loss: 3.5158
Epoch 2/30
93/93 [==============================] - 16s 169ms/step - loss: 2.8153
Epoch 3/30
93/93 [==============================] - 16s 171ms/step - loss: 2.7290
Epoch 4/30
93/93 [==============================] - 16s 172ms/step - loss: 2.6370
Epoch 5/30
93/93 [==============================] - 16s 173ms/step - loss: 2.5712
Epoch 6/30
93/93 [==============================] - 16s 175ms/step - loss: 2.5242
Epoch 7/30
93/93 [==============================] - 17s 177ms/step - loss: 2.4794
Epoch 8/30
93/93 [==============================] - 17s 179ms/step - loss: 2.4356
Epoch 9/30
93/93 [==============================] - 17s 181ms/step - loss: 2.3869
Epoch 10/30
93/93 [==============================] - 17s 182ms/step - loss: 2.3401
Epoch 11/30
93/93 [==============================] - 17s 182ms/step - loss: 2.2947
Epoch 12/30
93/93 [==============================] - 17s 183ms/step - loss: 2.2528
Epoch 13/30
9

## 3. 모델 평가하기

작문의 결과를 알고리즘이 평가하는 것은 무리라고 한다.
<br>
작문을 시켜보고 직접 평가해보자.

### 시작 문장을 전달하면 작문을 진행하는 함수를 생성

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### while문 왜why?

1. 학습 단계에서는 준비된 데이터셋(소스 문장과 타겟 문장)을 비교했다.
2. 목표는 문장을 생성하는 프로그램을 만드는 것이다. 즉, 테스트 데이터셋을 대상으로 하는 것이 아니다.
3. 반복문을 통해, 인자로 받은(혹은 디폴트인 \<strat>) 값을 기반으로, 다음 값을 예측한다.


- while문의 동작 설명은 코드 안의 주석

In [23]:
# 함수 내부의 생성 문장에 god을 추가해 실행해 본 결과
generate_text(model, tokenizer, init_sentence="<start> god")

'<start> god respected spider spider dull certainly pledge pledge currents autolycus hearing wisdom wisdom coast coast opening opening opening apothecary '

# 프로젝트 : '멋진' 작사가 만들기

### 데이터 준비

In [61]:
import glob

#코퍼스 데이터 경로
txt_file_path = path + 'lyricist/data/lyrics/*'

#glob으로 파일 위치 리스트 생성
txt_list = glob.glob(txt_file_path)

l_raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        l_raw_corpus.extend(raw)
        
print("데이터 크기:", len(l_raw_corpus))
print("Examples:\n", l_raw_corpus[:3])        

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


### 데이터 전처리

>```
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() 
    sentence = '\<start> ' + sentence + ' \<end>' 
    return sentence


In [62]:
for idx, sentence in enumerate(l_raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### 정규표현식으로 정제 후 리스트에 담기

In [67]:
l_corpus = []

for sentence in l_raw_corpus:
    
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    preprocessed_sentence = preprocess_sentence(sentence) #프리프로세스(..)함수는 예제와 동일하다.
    l_corpus.append(preprocessed_sentence)

l_corpus[:3]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>']

In [68]:
#정규표현식 정제 과정을 거치고 데이터의 양이 약간 감소한 것을 확인 할 수 있다.
len(l_corpus)

175749

- 비교를 위해 예제의 코드를 카피

>```
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="\<unk>"
    )   
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
>    
    print(tensor,tokenizer)
    return tensor, tokenizer
>

### 토큰화 (텐서 형태로 변환)

In [69]:

def l_tokenize(a):
    l_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  #단어장의 크기, 12000개 이상으로 설정.
        filters=' ',
        oov_token='<unk>'
    )
    
    l_tokenizer.fit_on_texts(a)
    l_tensor = l_tokenizer.texts_to_sequences(a)
    
    # 토큰화 후 텐서의 형태로 변환하기 전 토큰 갯수가 15를 초과하는 대상을 제외하는 과정
    ### print(type(l_tensor)) ###
    # 이 단계에서 l_tensor의 데이터 타입을 확인해보면 <class 'list'>로 나온다.
    # 일부를 출력해보면 텐서형태로 변환된 것과 유사해 보이지만, 리스트안의 리스트 형태이다.
    # 리스트의 리무브 메소드를 for문으로, 각 요소의 길이가 15개를 넘는 경우 리스트에서 제거한다.
    
    for cut in l_tensor:
        if len(cut) > 15:
            l_tensor.remove(cut)

    l_tensor = tf.keras.preprocessing.sequence.pad_sequences(l_tensor, maxlen=15, padding='post')
    
    print(l_tensor, l_tokenizer)
    return l_tensor, l_tokenizer

### 위의 tokenize코드(구분을 위해 변수명들에 l_을 붙여줌)에 대한 리뷰

> '문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기를 권합니다.'

- 세부적 내용을 다 학습하지는 못했지만, 전체적인 흐름에서 막힘 없이 진행되던<br> 이번 익스플로레이션이 가장 난항을 겪은 부분이다.


- 원하는 결과를 얻어내기까지 겪은 시행착오는 다음과 같다.
    1. 가장 먼저 떠올린 방법은 정규표현식의 코드를 건드려보는 것이었다.

        - 이 경우 정규표현식으로 정제하는 대상 데이터가 '문자열 타입이기 때문'에 <br>구현이 어렵다는 것을 바로 파악할 수 있었다.      
    2. 다음은 pad_sequences 메소드를 자세히 알아보는 것이었다.

        - maxlen 인자값을 받아 길이를 제한할 수 있다는 것을 알게 되었다.
            
        - 하지만 적용해보니 기대와 달리 데이터 shape의 열 길이가 줄어들지 않았다.
        
        - 그런데 15개 보다 긴 문장이 없었기 때문에 한참을 생각하게 되었다.
        
        - 각 단계에서의 shape, len, type등을 출력해보며 알게 된 사실은
            <br> 'maxlen'은 이미 텐서화 된 데이터의 최대 길이만 제한하는 파라미터라는 것이다.        
            
    3. 위의 착오를 겪으며 함수 안에 for문을 작성해주었다. 설명은 코드에 포함된 주석으로 대체.
   
    4. 마지막 문제는 for문을 넣으며 maxlen을 제거해서 발생했다.
        - l_tensor의 shape을 확인해보니 열 길이가 확실히 줄어들었다.
        - 그런데 행 길이가 이상했다. 결과는 아래와 같았다.
        > print(l_tensor.shape)
        > 
        >(159272, 347)
        - 이유는 remove 메소드의 특징에 있었다.
        - remove메소드는 리스트의 요소들을 제거하지만, 각 요소들의 자리값은 남아있다.
        - l_tensor은 리스트 안에 요소로 리스트가 들어있는 2차원 리스트이다.
        - for문을 통해 길이 15이상의 요소 리스트는 remove된다.
        - *정확한 이해인지는 모르겠지만* 비어있는 자리값들은 바로 앞 리스트에 합쳐진다. <br>아마 시퀀스 객체의 연속성을 유지하기 위한 방식이 아닐까 추측해보았다.
        - 결과적으로 347이라는 길이는 아래와 같이 추측된다.
        > 15 이하인 len(tensor)[n] 과
        >
        > 16 이상인 len(tensor)[n+1] + len(tensor)[n+2] + .. + len(tensor)[n+k]
        > 
        > 의 길이가 최대인 경우
        - 그리고 padding=post로 인해 모든 요소의 뒤에 최대 345개의 0이 포함된 텐서가 생성되었다.

- maxlen=15 를 다시 넣어준 결과 원하던 결과물을 확인할 수 있었다.

*최종적으로..*
>  1. 길이값이 15가 넘어 제거된 요소들의 길이가 앞 요소에 더해짐 
>  2. maxlen = 15에 의해 텐서의 길이가 제한됨 <br>(여기서 제거된 시퀀스의 뒷부분은 전부 토큰 인덱스 0으로의미가 없는 값임)
>  3. 토큰 갯수 15개 이상인 데이터를 제외한 데이터만 텐서로 반환됨

In [70]:
# 2로 시작하고 있으니 정상적으로 진행되고 있는 것 같다.
l_tensor, l_tokenizer = l_tokenize(l_corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   2  261  200 ...   12    3    0]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f66b26a5340>


### 텐서 데이터를 객체화

In [71]:
#소스 문장과 타겟 문장 생성
src_input = l_tensor[:, :-1]
tgt_input = l_tensor[:, 1:]

In [72]:
#타겟 문장이 2로 시작하지 않고 한 칸씩 밀려있는 것을 확인
print(src_input[0:5])
print(tgt_input[0:5])

[[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0    0    0]
 [   2   36    7   37   15  164  282   28  299    4   47    7   43    3]
 [   2   11  354   25   42    3    0    0    0    0    0    0    0    0]
 [   2    6 3604    4    6 2265    3    0    0    0    0    0    0    0]]
[[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]
 [  17 2639  873    4    8   11 6043    6  329    3    0    0    0    0]
 [  36    7   37   15  164  282   28  299    4   47    7   43    3    0]
 [  11  354   25   42    3    0    0    0    0    0    0    0    0    0]
 [   6 3604    4    6 2265    3    0    0    0    0    0    0    0    0]]


In [102]:
#데이터셋 객체 생성은 sklearn의 train_test_split() 메소드로 진행
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=123)

In [74]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127417, 14)
Target Train: (127417, 14)


In [89]:
# 모델의 정의는 아직 손 댈 영역이 아닌 것 같다. 그대로 사용하자
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(l_tokenizer.num_words + 1, embedding_size , hidden_size)

In [90]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

In [91]:
# 1 batch 만큼의 임시 모델을 확인해보자
for src_sample, tgt_sample in dataset.take(1): break

In [92]:
model(src_sample)

<tf.Tensor: shape=(256, 20, 12001), dtype=float32, numpy=
array([[[ 4.22118959e-04,  2.94097612e-04, -1.57962917e-04, ...,
          1.80483374e-04, -1.41656012e-04,  1.04516999e-04],
        [ 6.67420274e-04,  3.47259192e-04, -1.23224178e-04, ...,
          2.77022627e-04, -3.84744722e-04,  3.65340442e-04],
        [ 6.16616104e-04,  2.55826395e-04, -2.04375130e-04, ...,
          3.15672834e-04, -4.39194526e-04,  5.73804893e-04],
        ...,
        [ 1.84376759e-03,  3.39808920e-03,  3.44194268e-04, ...,
         -1.39116088e-03, -2.68435944e-03,  2.99307867e-04],
        [ 1.77112559e-03,  3.54930805e-03,  4.34379675e-04, ...,
         -1.58158015e-03, -2.75665359e-03,  2.30596735e-04],
        [ 1.68531516e-03,  3.67000024e-03,  5.22705377e-04, ...,
         -1.74139964e-03, -2.81358347e-03,  1.69815103e-04]],

       [[ 4.22118959e-04,  2.94097612e-04, -1.57962917e-04, ...,
          1.80483374e-04, -1.41656012e-04,  1.04516999e-04],
        [ 7.09087064e-04,  3.78239405e-04, -4

In [93]:
model.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_12 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_13 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_6 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


### 학습!
#### 그리고 두 번째 난관이었다고 한다.

- 예제에서 사용한 학습 모델은 아래와 같았다
> dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset


- 문제는 프로젝트 실습의 경우 사이킷런으로 스플릿 한 데이터를 바로 학습시킨다는 것이었다.


- 그 자체는 어려울 것 없이 오히려 익숙했지만, 기본적으로 주어진 코드가 문제였다
> enc_train, enc_val, dec_train, dec_val = 


- 문장을 생성하는 모델은 test데이터에 대한 검증이 아니다.


- 그리고 총체적인 기반지식의 부족.
    - 테스트 데이터에 해당하는 분류는 어디에 쓰이는 것인가?
    - 밸리데이션 용도로 추정되는데 어떻게 사용하는 것인가?
    - train_test_split의 파라미터는 train-Validation 의 경우와 train-test 의 경우 다른가?


결과적으로 한 epoch의 학습 당 검증까지 한 번에 가능하다는 (어쩌면 당연한)지식을 습득한 후 해결이 되었으나, 큰 인지부조화를 겪어 해결까지 오랜 시간이 걸렸다.

In [ ]:
# 10epoch안에 val_loss값을 2.2 수준으로 줄이는 것이 목표.
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [103]:
# 예제에 주어진 하이퍼파라미터들을 그대로 사용한 결과 val_loss가 2.5 정도에서 멈췄다.
# hidden_size는 일꾼의 수..? 일꾼을 늘려보자.
# 또 2.2에 도달하지 못해 batch_size를 조절해보았다.
# 128에서 256으로 배치사이즈를 키웠을 때, train데이터의 로스율은 상당히 개선되었으나 val_loss는 감소폭이 매우낮았음
# 반대로 배치사이즈를 줄여 학습의 간격을 좁혀보았다.
# 지속적인 실패에, 노래 가사에는 후렴 반복 등이 많다는 생각이 들었다. 스플릿 과정에 랜덤스테이트를 넣어봄
embedding_size = 256
hidden_size = 2048
model.fit(enc_train, dec_train, epochs=10, batch_size=128, 
          validation_data = (enc_val, dec_val))

Epoch 1/10
996/996 [==============================] - 104s 105ms/step - loss: 1.3549 - val_loss: 2.2888
Epoch 2/10
996/996 [==============================] - 105s 105ms/step - loss: 1.2003 - val_loss: 2.0331
Epoch 3/10
996/996 [==============================] - 105s 105ms/step - loss: 1.1250 - val_loss: 1.8469
Epoch 4/10
996/996 [==============================] - 104s 104ms/step - loss: 1.0720 - val_loss: 1.7044
Epoch 5/10
996/996 [==============================] - 104s 105ms/step - loss: 1.0304 - val_loss: 1.5921
Epoch 6/10
996/996 [==============================] - 105s 105ms/step - loss: 0.9975 - val_loss: 1.5087
Epoch 7/10
996/996 [==============================] - 104s 104ms/step - loss: 0.9706 - val_loss: 1.4450
Epoch 8/10
996/996 [==============================] - 104s 104ms/step - loss: 0.9500 - val_loss: 1.3998
Epoch 9/10
996/996 [==============================] - 104s 105ms/step - loss: 0.9346 - val_loss: 1.3677
Epoch 10/10
996/996 [==============================] - 105s 105m

### 한 곡조 뽑아 보거라

In [106]:
generate_text(model, l_tokenizer, init_sentence="<start> world")

'<start> world is gettin ready , everybody s ready , yeah ! <end> '

*세상은 준비되고 있고, 모두들 준비됐어, 그래!*

만족스러운 무대였습니다. special thx PAPAGO

## 마치며
- 비단 익스플로래이션을 진행 할 때가 아니더라도, 의외의 복병에 시간을 빼앗기는 경우가 많다.
- 이번 과정에서는 물론 얻은 것이 적지 않다고 생각한다. 
- 하지만 항상 그럴 수 있는 것은 아니니 삼천포로 빠지지 않고, 맺고 끊는 것을 잘 해야 할 것 같다.
- 최종적인 목표..에 어느 것이 더 낫다고 할 수는 없겠지만 이 상태로라면<br> 
    아이펠 과정을 소화하는 것 자체가 무리일 것 같다.


### 기억에 남는 학습 내용
- 리스트의 remove 메서드.
- train_test_split에서 밸리데이션을 한 번에 돌릴 수 있는 인자.
- 스플릿 과정에서 랜덤스테이트를 넣었을 때 나타난 드라마틱한 차이.
 > 한 인간이 쓰는 단어는 한정적이고, 그 글의 전문성이 낮을 수록 더 심하다.
 >
 > 심지어 노래가사가 원본데이터라면... 당연히 섞어줬어야 했다.


### 어려웠던 점
이번 익스는 이후에 배우게 될 것들이 앞서 나오면서, 우선 넘어가자는 부분이 많았는데
그런 부분에서 혼란을 많이 겪은 것 같다. 효율의 측면에서, 문제상황에 직면했을 때, 그 수준이 낮더라도 
나만의 해결해 나가는 솔루션을 어느 정도 정형화 할 필요가 느껴졌다. 



### 추가로 해보고 싶은 점
- 지금 당장은 넘모 피곤해서 없다. remove메소드를 시작으로 시퀀스 자료형의 특징들을 공부해봐야겠다.


### 총평
: 느무..느무느무... 졸립니다..
아이펠 화이팅.. 나.. 화이팅...
